In [ ]:
import scipy.io
data = scipy.io.loadmat('Data_ankle.mat')
ankle_data= data['ankle_data']
print(ankle_data.shape)

(48000, 7)


# New Section

In [ ]:
import pandas as pd
# Define column names
column_names = [
    'time',
    'vgrf',  # vertical ground reaction force
    'trunk_angle',
    'hip_angle',
    'knee_angle',
    'ankle_angle',
    'joint_moment'
]


In [ ]:
df = pd.DataFrame(ankle_data, columns=column_names )
df.head()

,time,vgrf,trunk_angle,hip_angle,knee_angle,ankle_angle,joint_moment
0,0.00,-10.321381,-0.094286,0.292974,0.825063,-1.492588,-2.011628
1,0.01,-7.990312,-0.096174,0.293179,0.822469,-1.493192,-3.655209
2,0.02,-6.208738,-0.096449,0.294008,0.810997,-1.495867,-5.786433
3,0.03,-5.126637,-0.097196,0.295568,0.785731,-1.501793,-6.190008
4,0.04,-4.774037,-0.095624,0.297472,0.745707,-1.511208,-5.542868


In [ ]:
# Split the data into training and testing sets
train_size = int(0.8 * len(df))
train_data = df[:train_size]
test_data = df[train_size:]

In [ ]:
# Separate the features and target variable
X_train = train_data.drop(columns=['time', 'joint_moment'])
Y_train = train_data['joint_moment'].values.reshape(-1, 1)  # Reshape for scaler
X_test = test_data.drop(columns=['time', 'joint_moment'])
Y_test = test_data['joint_moment'].values.reshape(-1, 1)  # Reshape for scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
# Apply scaling only to the training data
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)  # Use the same scaler to transform the test data

In [ ]:
# Apply scaling to the target variable
scaler_Y = StandardScaler()
Y_train_scaled = scaler_Y.fit_transform(Y_train)  # Fit and transform the training target variable
Y_test_scaled = scaler_Y.transform(Y_test)  # Transform the test target variable using the same scaler

print(Y_test_scaled)

[[-0.98351688]
 [-0.98411872]
 [-0.96365408]
 ...
 [ 0.30780874]
 [ 0.34680355]
 [ 0.37615492]]


In [ ]:
# Convert scaled data back to DataFrame
normalized_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
normalized_train_df['time'] = train_data['time'].reset_index(drop=True)
normalized_train_df['joint_moment'] = Y_train_scaled.flatten()  # Flatten to 1D

normalized_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
normalized_test_df['time'] = test_data['time'].reset_index(drop=True)
normalized_test_df['joint_moment'] = Y_test_scaled.flatten()  # Flatten to 1D

In [ ]:
# Print the normalized data
print(train_data.head())
print(test_data.head())

# If you need to access the target variable from the test data separately
target_Ytest_normalized = normalized_test_df['joint_moment']
print(target_Ytest_normalized.head())

   time       vgrf  trunk_angle  hip_angle  knee_angle  ankle_angle  \
0  0.00 -10.321381    -0.094286   0.292974    0.825063    -1.492588   
1  0.01  -7.990312    -0.096174   0.293179    0.822469    -1.493192   
2  0.02  -6.208738    -0.096449   0.294008    0.810997    -1.495867   
3  0.03  -5.126637    -0.097196   0.295568    0.785731    -1.501793   
4  0.04  -4.774037    -0.095624   0.297472    0.745707    -1.511208   

   joint_moment  
0     -2.011628  
1     -3.655209  
2     -5.786433  
3     -6.190008  
4     -5.542868  
         time       vgrf  trunk_angle  hip_angle  knee_angle  ankle_angle  \
38400  384.00 -11.512108    -0.032512   0.007387    0.973938    -1.273562   
38401  384.01 -15.839764    -0.035124   0.037376    1.022451    -1.272575   
38402  384.02 -16.915961    -0.038637   0.066333    1.062992    -1.280968   
38403  384.03 -15.981315    -0.044385   0.093862    1.094613    -1.295619   
38404  384.04 -13.927180    -0.048895   0.119811    1.116854    -1.313351   

  

In [ ]:
def extract_data_for_combination(columns):
    X_train = normalized_train_df[columns].values
    y_train = normalized_train_df['joint_moment'].values
    X_test = normalized_test_df[columns].values
    y_test = normalized_test_df['joint_moment'].values

    # Reshape for RNN
    X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    # Reshape for CNN
    X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    return X_train_rnn, y_train, X_test_rnn, y_test, X_train_cnn, X_test_cnn


In [ ]:
from keras.models import Sequential
from keras.layers import  Dense, Dropout , LSTM ,  BatchNormalization

# RNN Model creation function
def create_rnn_model(input_shape, units, dropout_rate):
    model = Sequential()

    # First LSTM layer
    model.add(LSTM(units, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    # Second LSTM layer
    model.add(LSTM(units, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Batch Normalization
    model.add(BatchNormalization())

    # Dense layer
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Flatten, Dropout, BatchNormalization

# CNN Model creation function
def create_cnn_model(input_shape, units, dropout_rate):
    model = Sequential()

    # First Conv1D layer with kernel size 1
    model.add(Conv1D(filters=units, kernel_size=1, activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    # Second Conv1D layer with kernel size 1
    model.add(Conv1D(filters=units, kernel_size=1, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten layer
    model.add(Flatten())

    # Batch Normalization
    model.add(BatchNormalization())

    # Dense layer
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    return model



In [ ]:
from sklearn.metrics import mean_squared_error
import os

def train_and_evaluate_rnn_model(input_comb, model_path):
    X_train_rnn, y_train, X_test_rnn, y_test, _, _ = extract_data_for_combination(input_comb)
    rnn_model = create_rnn_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]))
    history = rnn_model.fit(X_train_rnn, y_train, epochs=53, batch_size=64, validation_split=0.2, verbose=1)
    predictions = rnn_model.predict(X_test_rnn).flatten()
    mse = mean_squared_error(y_test, predictions)
    rnn_model.save(model_path)
    return mse, predictions, history

In [ ]:
def train_and_evaluate_cnn_model(input_comb, model_path):
    _, y_train, _, y_test, X_train_cnn, X_test_cnn = extract_data_for_combination(input_comb)
    cnn_model = create_cnn_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
    history = cnn_model.fit(X_train_cnn, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)
    predictions = cnn_model.predict(X_test_cnn).flatten()
    mse = mean_squared_error(y_test, predictions)
    cnn_model.save(model_path)
    return mse, predictions, history

In [ ]:
# Directory to save the models
model_dir = 'models/'
os.makedirs(model_dir, exist_ok=True)

# Define input combinations
input_combinations = {
    'all_angles_and_vgrf': ['vgrf', 'trunk_angle', 'hip_angle', 'knee_angle', 'ankle_angle'],
    'ankle_angle_only': ['vgrf', 'ankle_angle'],
    'all_angles': ['trunk_angle', 'hip_angle', 'knee_angle', 'ankle_angle'],
}

results = {}
for key, columns in input_combinations.items():

      # CNN Model
    cnn_model_path = os.path.join(model_dir, f"cnn_model_{key}.h5")
    cnn_mse, cnn_predictions, cnn_history = train_and_evaluate_cnn_model(columns, cnn_model_path)

    # RNN Model
    rnn_model_path = os.path.join(model_dir, f"rnn_model_{key}.h5")
    rnn_mse, rnn_predictions, rnn_history = train_and_evaluate_rnn_model(columns, rnn_model_path)



    results[key] = {
        'rnn_mse': rnn_mse,
        'rnn_predictions': rnn_predictions,
        'rnn_history': rnn_history,
        'cnn_mse': cnn_mse,
        'cnn_predictions': cnn_predictions,
        'cnn_history': cnn_history
    }

    print(f"Input Combination: {key}")
    print(f"RNN - MSE: {rnn_mse}")
    print(f"CNN - MSE: {cnn_mse}")


Epoch 1/50
480/480 [==============================] - 6s 7ms/step - loss: 0.1518 - val_loss: 0.0612
Epoch 2/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0512 - val_loss: 0.0230
Epoch 3/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0394 - val_loss: 0.0184
Epoch 4/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0344 - val_loss: 0.0286
Epoch 5/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0309 - val_loss: 0.0149
Epoch 6/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0279 - val_loss: 0.0208
Epoch 7/50
480/480 [==============================] - 3s 7ms/step - loss: 0.0257 - val_loss: 0.0151
Epoch 8/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0241 - val_loss: 0.0147
Epoch 9/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0229 - val_loss: 0.0127
Epoch 10/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0219 - val_loss: 0.0139

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/53
480/480 [==============================] - 6s 5ms/step - loss: 0.0844 - val_loss: 0.4249
Epoch 2/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0520 - val_loss: 0.0160
Epoch 3/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0468 - val_loss: 0.0147
Epoch 4/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0463 - val_loss: 0.0108
Epoch 5/53
480/480 [==============================] - 2s 5ms/step - loss: 0.0452 - val_loss: 0.0108
Epoch 6/53
480/480 [==============================] - 3s 7ms/step - loss: 0.0445 - val_loss: 0.0105
Epoch 7/53
480/480 [==============================] - 3s 6ms/step - loss: 0.0423 - val_loss: 0.0098
Epoch 8/53
480/480 [==============================] - 2s 5ms/step - loss: 0.0410 - val_loss: 0.0085
Epoch 9/53
480/480 [==============================] - 2s 5ms/step - loss: 0.0401 - val_loss: 0.0085
Epoch 10/53
480/480 [==============================] - 3s 6ms/step - loss: 0.0391 - val_loss: 0.0075

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Input Combination: all_angles_and_vgrf
RNN - MSE: 0.008007969303112351
CNN - MSE: 0.01038276997141023
Epoch 1/50
480/480 [==============================] - 3s 4ms/step - loss: 0.2006 - val_loss: 0.1812
Epoch 2/50
480/480 [==============================] - 2s 3ms/step - loss: 0.1374 - val_loss: 0.0995
Epoch 3/50
480/480 [==============================] - 1s 3ms/step - loss: 0.1240 - val_loss: 0.0871
Epoch 4/50
480/480 [==============================] - 2s 3ms/step - loss: 0.1157 - val_loss: 0.0843
Epoch 5/50
480/480 [==============================] - 2s 3ms/step - loss: 0.1076 - val_loss: 0.0853
Epoch 6/50
480/480 [==============================] - 2s 4ms/step - loss: 0.1032 - val_loss: 0.0806
Epoch 7/50
480/480 [==============================] - 2s 5ms/step - loss: 0.1019 - val_loss: 0.0956
Epoch 8/50
480/480 [==============================] - 2s 3ms/step - loss: 0.1003 - val_loss: 0.0811
Epoch 9/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0976 - val_loss: 0.085

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
480/480 [==============================] - 3s 4ms/step - loss: 0.1407 - val_loss: 0.1318
Epoch 2/50
480/480 [==============================] - 3s 5ms/step - loss: 0.0545 - val_loss: 0.0365
Epoch 3/50
480/480 [==============================] - 2s 5ms/step - loss: 0.0447 - val_loss: 0.0358
Epoch 4/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0390 - val_loss: 0.0303
Epoch 5/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0352 - val_loss: 0.0197
Epoch 6/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0326 - val_loss: 0.0176
Epoch 7/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0306 - val_loss: 0.0182
Epoch 8/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0293 - val_loss: 0.0164
Epoch 9/50
480/480 [==============================] - 3s 6ms/step - loss: 0.0280 - val_loss: 0.0158
Epoch 10/50
480/480 [==============================] - 2s 4ms/step - loss: 0.0266 - val_loss: 0.0178

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/53
480/480 [==============================] - 5s 5ms/step - loss: 0.0903 - val_loss: 0.4294
Epoch 2/53
480/480 [==============================] - 3s 7ms/step - loss: 0.0566 - val_loss: 0.0167
Epoch 3/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0548 - val_loss: 0.0155
Epoch 4/53
480/480 [==============================] - 2s 5ms/step - loss: 0.0492 - val_loss: 0.0155
Epoch 5/53
480/480 [==============================] - 2s 5ms/step - loss: 0.0481 - val_loss: 0.0140
Epoch 6/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0480 - val_loss: 0.0131
Epoch 7/53
480/480 [==============================] - 3s 5ms/step - loss: 0.0450 - val_loss: 0.0142
Epoch 8/53
480/480 [==============================] - 3s 6ms/step - loss: 0.0450 - val_loss: 0.0126
Epoch 9/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0471 - val_loss: 0.0123
Epoch 10/53
480/480 [==============================] - 2s 4ms/step - loss: 0.0447 - val_loss: 0.0125

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.7 MB/s eta 0:00:00


In [ ]:
import os
import optuna
import optuna.visualization as vis
import matplotlib.pyplot as plt


In [ ]:
def objective(trial, model_type, input_comb):
    # Suggest hyperparameters for both RNN and CNN
    units = trial.suggest_int('units', 10, 100)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    epochs = trial.suggest_int('epochs', 10, 50)
    batch_size = trial.suggest_int('batch_size', 16, 128)

    # Extract data for the input combination
    X_train_rnn, y_train, X_test_rnn, y_test, X_train_cnn, X_test_cnn = extract_data_for_combination(input_comb)

    if model_type == 'rnn':
        model = create_rnn_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), units=units, dropout_rate=dropout_rate)
        history = model.fit(X_train_rnn, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        predictions = model.predict(X_test_rnn).flatten()

    elif model_type == 'cnn':
        model = create_cnn_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]), units=units, dropout_rate=dropout_rate)
        history = model.fit(X_train_cnn, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        predictions = model.predict(X_test_cnn).flatten()

    mse = mean_squared_error(y_test, predictions)
    return mse


In [ ]:
# Function to optimize model using Optuna
def optimize_model(model_type, input_comb_key, input_comb, n_trials=10):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, model_type, input_comb), n_trials=n_trials)

    best_trial = study.best_trial
    print(f"Best trial for {model_type} with input {input_comb_key}:")
    print(f"  Value: {best_trial.value}")
    print(f"  Params: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")

    # Generate and display Optuna visualizations
    vis.plot_optimization_history(study).show()
    vis.plot_param_importances(study).show()

    return best_trial


In [ ]:
# Define input combinations
input_combinations = {
    'all_angles_and_vgrf': ['vgrf', 'trunk_angle', 'hip_angle', 'knee_angle', 'ankle_angle'],
    'ankle_angle_only': ['vgrf', 'ankle_angle'],
    'all_angles': ['trunk_angle', 'hip_angle', 'knee_angle', 'ankle_angle'],
}

results = {}
for key, columns in input_combinations.items():
    print(f"Optimizing RNN for input combination: {key}")
    rnn_best_trial = optimize_model('rnn', key, columns, n_trials=10)

    print(f"Optimizing CNN for input combination: {key}")
    cnn_best_trial = optimize_model('cnn', key, columns, n_trials=10)

    results[key] = {
        'rnn_best_trial': rnn_best_trial,
        'cnn_best_trial': cnn_best_trial
    }

print(results)

[I 2024-07-15 16:11:15,054] A new study created in memory with name: no-name-4ad38ee4-9b6d-4498-89ff-5065cf806893


Optimizing RNN for input combination: all_angles_and_vgrf
300/300 [==============================] - 1s 3ms/step


[I 2024-07-15 16:12:00,649] Trial 0 finished with value: 0.008930224409600566 and parameters: {'units': 99, 'dropout_rate': 0.24179170755847235, 'epochs': 13, 'batch_size': 104}. Best is trial 0 with value: 0.008930224409600566.


300/300 [==============================] - 1s 2ms/step


[I 2024-07-15 16:14:27,323] Trial 1 finished with value: 0.008427449717145962 and parameters: {'units': 66, 'dropout_rate': 0.27675331317142843, 'epochs': 46, 'batch_size': 77}. Best is trial 1 with value: 0.008427449717145962.


...

[I 2024-07-15 17:35:42,236] Trial 8 finished with value: 0.02127120517950852 and parameters: {'units': 79, 'dropout_rate': 0.4811023032123547, 'epochs': 16, 'batch_size': 32}. Best is trial 7 with value: 0.01541162879811736.


300/300 [==============================] - 1s 2ms/step


[I 2024-07-15 17:36:07,766] Trial 9 finished with value: 0.028333727350368144 and parameters: {'units': 53, 'dropout_rate': 0.44348839042804433, 'epochs': 13, 'batch_size': 92}. Best is trial 7 with value: 0.01541162879811736.


Best trial for cnn with input all_angles:
  Value: 0.01541162879811736
  Params: 
    units: 88
    dropout_rate: 0.4489498860037714
    epochs: 33
    batch_size: 55


{'all_angles_and_vgrf': {'rnn_best_trial': FrozenTrial(number=9, state=TrialState.COMPLETE, values=[0.007542293262754615], datetime_start=datetime.datetime(2024, 7, 15, 16, 25, 51, 11950), datetime_complete=datetime.datetime(2024, 7, 15, 16, 27, 17, 728359), params={'units': 70, 'dropout_rate': 0.35241529851814357, 'epochs': 28, 'batch_size': 104}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'units': IntDistribution(high=100, log=False, low=10, step=1), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'epochs': IntDistribution(high=50, log=False, low=10, step=1), 'batch_size': IntDistribution(high=128, log=False, low=16, step=1)}, trial_id=9, value=None), 'cnn_best_trial': FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.011235552231079722], datetime_start=datetime.datetime(2024, 7, 15, 16, 32, 30, 683119), datetime_complete=datetime.datetime(2024, 7, 15, 16, 35, 55, 572837), params={'units': 94, 'dropout_rate': 0.18441235

In [ ]:
print(results)

{'all_angles_and_vgrf': {'rnn_best_trial': FrozenTrial(number=9, state=TrialState.COMPLETE, values=[0.007542293262754615], datetime_start=datetime.datetime(2024, 7, 15, 16, 25, 51, 11950), datetime_complete=datetime.datetime(2024, 7, 15, 16, 27, 17, 728359), params={'units': 70, 'dropout_rate': 0.35241529851814357, 'epochs': 28, 'batch_size': 104}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'units': IntDistribution(high=100, log=False, low=10, step=1), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'epochs': IntDistribution(high=50, log=False, low=10, step=1), 'batch_size': IntDistribution(high=128, log=False, low=16, step=1)}, trial_id=9, value=None), 'cnn_best_trial': FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.011235552231079722], datetime_start=datetime.datetime(2024, 7, 15, 16, 32, 30, 683119), datetime_complete=datetime.datetime(2024, 7, 15, 16, 35, 55, 572837), params={'units': 94, 'dropout_rate': 0.18441235